In [18]:
import pandas
import sklearn
from sklearn.cluster import KMeans
import numpy as np

In [5]:
df = pandas.read_csv("menu.csv")
print(df)

               Category                                               Item  \
0             Breakfast                                       Egg McMuffin   
1             Breakfast                                  Egg White Delight   
2             Breakfast                                   Sausage McMuffin   
3             Breakfast                          Sausage McMuffin with Egg   
4             Breakfast                   Sausage McMuffin with Egg Whites   
..                  ...                                                ...   
255  Smoothies & Shakes                 McFlurry with Oreo Cookies (Small)   
256  Smoothies & Shakes                McFlurry with Oreo Cookies (Medium)   
257  Smoothies & Shakes                 McFlurry with Oreo Cookies (Snack)   
258  Smoothies & Shakes  McFlurry with Reese's Peanut Butter Cups (Medium)   
259  Smoothies & Shakes   McFlurry with Reese's Peanut Butter Cups (Snack)   

        Serving Size  Calories  Calories from Fat  Total Fat  \

In [6]:
labels = []
for c in df['Category']:
    labels.append(c)

In [7]:
print( labels)

['Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Breakfast', 'Beef & Pork', 'Beef & Pork', 'Beef & Pork', 'Beef & Pork', 'Beef & Pork', 'Beef & Pork', 'Beef & Pork', 'Beef & Pork', 'Beef & Pork', 'Beef & Pork', 'Beef & Pork', 'Beef & Pork', 'Beef & Pork', 'Beef & Pork', 'Beef & Pork', 'Chicken & Fish', 'Chicken & Fish', 'Chicken & Fish', 'Chicken & Fish', 'Chicken & Fish', 'Chicken & Fish', 'Chicken & Fish', 'Chicken & Fish', 'Chicken & Fish', 'Chicken & Fish', 'Chicken & Fish', 'Chicken & Fish', 'Chicken & F

Delete category and item columns

In [8]:
df.pop("Category")
df.pop("Item")
df.pop("Serving Size")

0       4.8 oz (136 g)
1       4.8 oz (135 g)
2       3.9 oz (111 g)
3       5.7 oz (161 g)
4       5.7 oz (161 g)
            ...       
255    10.1 oz (285 g)
256    13.4 oz (381 g)
257     6.7 oz (190 g)
258    14.2 oz (403 g)
259     7.1 oz (202 g)
Name: Serving Size, Length: 260, dtype: object

In [9]:
print(df)

     Calories  Calories from Fat  Total Fat  Total Fat (% Daily Value)  \
0         300                120       13.0                         20   
1         250                 70        8.0                         12   
2         370                200       23.0                         35   
3         450                250       28.0                         43   
4         400                210       23.0                         35   
..        ...                ...        ...                        ...   
255       510                150       17.0                         26   
256       690                200       23.0                         35   
257       340                100       11.0                         17   
258       810                290       32.0                         50   
259       410                150       16.0                         25   

     Saturated Fat  Saturated Fat (% Daily Value)  Trans Fat  Cholesterol  \
0              5.0                

In [10]:
df_normalized = ((df-df.min())/(df.max()-df.min()))*10

In [11]:
print(df_normalized)

     Calories  Calories from Fat  Total Fat  Total Fat (% Daily Value)  \
0    1.595745           1.132075   1.101695                   1.098901   
1    1.329787           0.660377   0.677966                   0.659341   
2    1.968085           1.886792   1.949153                   1.923077   
3    2.393617           2.358491   2.372881                   2.362637   
4    2.127660           1.981132   1.949153                   1.923077   
..        ...                ...        ...                        ...   
255  2.712766           1.415094   1.440678                   1.428571   
256  3.670213           1.886792   1.949153                   1.923077   
257  1.808511           0.943396   0.932203                   0.934066   
258  4.308511           2.735849   2.711864                   2.747253   
259  2.180851           1.415094   1.355932                   1.373626   

     Saturated Fat  Saturated Fat (% Daily Value)  Trans Fat  Cholesterol  \
0              2.5                

K-Means Clustering

In [12]:
X = df.values

In [13]:
print(X)

[[300. 120.  13. ...   0.  25.  15.]
 [250.  70.   8. ...   0.  25.   8.]
 [370. 200.  23. ...   0.  25.  10.]
 ...
 [340. 100.  11. ...   0.  25.   6.]
 [810. 290.  32. ...   0.  60.   6.]
 [410. 150.  16. ...   0.  30.   4.]]


In [14]:
kmeans = KMeans(n_clusters=5, init='k-means++')
kmeans.fit(X)

labels = kmeans.labels_
score = sklearn.metrics.davies_bouldin_score(X, labels)
print(score)

0.6976256239169594


kmeans++

In [15]:
kmeans = KMeans(n_clusters=5, n_init=1, max_iter=1, init='k-means++')

In [34]:
def train_kmeans(X, init):
    kmeans = KMeans(n_clusters=5, n_init=1, max_iter=1, init=init)

    state = None
    i = 0
    ys = []
    while True:
        kmeans.fit(X)

        labels = kmeans.labels_
        centroids = kmeans.cluster_centers_

        if np.array_equal(state, labels):
            break

        kmeans.init = centroids
        state = labels

        inertia = kmeans.inertia_
        score = sklearn.metrics.davies_bouldin_score(X, labels)
        i += 1

        ys.append(score)
#         print(i, inertia, score)

#     print('-' * 100, end='\n\n')

    return ys

In [26]:
train_kmeans(X, 'k-means++')

1 11724905.736692376 0.699398357251808
2 11663997.396907233 0.6933539211481247
3 11566136.892143209 0.6953525956720304
4 11557806.742300274 0.699567501219361
5 11552305.869625289 0.6976256239169594
----------------------------------------------------------------------------------------------------



[0.699398357251808,
 0.6933539211481247,
 0.6953525956720304,
 0.699567501219361,
 0.6976256239169594]

In [27]:
train_kmeans(X, 'random')

1 20697578.248429056 0.8513791817430512
2 19186154.708938945 0.8381212721445028
3 18167078.36242689 0.8268515261647675
4 17583540.39322839 0.8430062308767543
5 17212644.814345952 0.8523358679607693
6 16958839.276359964 0.8640469291318265
7 16893043.092079848 0.8714963364538229
8 16772599.512663906 0.8668101022331577
9 16669193.156726787 0.8660357636912668
10 16638218.024549466 0.8622421388792227
11 16624423.230999004 0.8648718122760899
----------------------------------------------------------------------------------------------------



[0.8513791817430512,
 0.8381212721445028,
 0.8268515261647675,
 0.8430062308767543,
 0.8523358679607693,
 0.8640469291318265,
 0.8714963364538229,
 0.8668101022331577,
 0.8660357636912668,
 0.8622421388792227,
 0.8648718122760899]

In [28]:
init = np.random.uniform(0.0, 1.0, (5, X.shape[1]))
train_kmeans(X, init)

1 44762279.20005958 0.6232095915087356
2 20636470.125200577 0.6789436339176638
3 15101451.925274387 0.6995856823188136
4 13993912.320370758 0.7106997803349117
5 13426749.458040044 0.6999296499533985
6 13074449.025940085 0.6893723012082208
7 12983211.98812861 0.6838023108635306
8 12923015.551917996 0.6787922816811945
9 12897816.409781646 0.6734130253456383
10 12876571.575393353 0.6694062188040403
11 12872061.206620105 0.6677564959026897
----------------------------------------------------------------------------------------------------



[0.6232095915087356,
 0.6789436339176638,
 0.6995856823188136,
 0.7106997803349117,
 0.6999296499533985,
 0.6893723012082208,
 0.6838023108635306,
 0.6787922816811945,
 0.6734130253456383,
 0.6694062188040403,
 0.6677564959026897]

In [51]:
plus_plus_max_iter = 0
plus_plus_results = []
plus_plus_mean = 0
plus_plus_std = 0
for i in range(50):
    ys = train_kmeans(X, 'k-means++')
    plus_plus_results.append(xs)
    xs = len(ys)
    max_iter = max(plus_plus_max_iter, xs)
    
plus_plus_mean = np.mean(plus_plus_results, axis=0)
plus_plus_std = np.std(plus_plus_results, axis=0)

In [52]:
random_max_iter = 0
random_results = []
random_mean = 0
random_std = 0
for i in range(50):
    ys = train_kmeans(X, 'k-means++')
    random_results.append(xs)
    xs = len(ys)
    max_iter = max(random_max_iter, xs)
    
random_mean = np.mean(plus_plus_results, axis=0)
random_std = np.std(plus_plus_results, axis=0)

In [53]:
total_random_max_iter = 0
total_random_results = []
total_random_mean = 0
total_random_std = 0
for i in range(50):
    ys = train_kmeans(X, 'k-means++')
    total_random_results.append(xs)
    xs = len(ys)
    max_iter = max(total_random_max_iter, xs)

total_random_mean = np.mean(plus_plus_results, axis=0)
total_random_std = np.std(plus_plus_results, axis=0)